#### Events Initial Analysis

The purpose of this notebook is to carry out a very rough, baseline analysis of the impact of events on nearby crimes. 

This analysis is limited in several important ways. See technical log for longer discussion, but to sum up some of the major ones:

- It uses single-day events, _without_ incorporating multi-day events from the dataset. This effectively intermingles our control (no-event) and treatment (event) groups, making it harder for us to detect a true effect.

- It uses a hard 1km square buffer around venues to identify possibly related crimes. Using this rather than a 'smart' buffer based on behavior also may increase the extent to which we intermingle impacted and unimpacted crimes, making it harder to detect a true effect. Also, if the buffer is too large, that will make it harder to detect a true effect. Conversely, if the buffer is too small we will increase the variance of our estimate.

- It relies on linear models. This isn't the worst thing ever but could _possibly_ be improved upon with a more nonlinear ML-y approach.

- It assumes that spillovers are (on average) linearly additive: being within the bandwidth of two events has on average 2x the effect of being within the bandwidth of one event.

- It relies on a selection-on-observables approach that only incorporates date, venue, and time as controls. This probably accounts for some significant part of any selection effect (day of the week, etc.) but not all. Other factors that are correlated with whether events take place and with crime will induce bias.

In [34]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import datetime as dt
from itertools import product
from sklearn import linear_model
import statsmodels.api as sm
%matplotlib inline
plt.rcParams['figure.figsize'] = [12, 12]  # add more space to figures

In [35]:
events = pd.DataFrame(gpd.read_file('geo/single_day_events.shp').drop(columns=['geometry']))
# convert event dates to datetime
events['date'] = pd.to_datetime(events['Date(s)'])
events

,Name of Ev,Category,Date(s),Location,Estimated,Number Peo,Address,Venue Capa,Venue Type,Latitude,Longitude,date
0,Alison Wonderland,Music,2018-08-24 00:00:00,Aragon Ballroom,None,4800,"1106 W Lawrence Ave, Chicago, IL 60640",4800.0,Event Space,41.969436,-87.658038,2018-08-24
1,"Virtual Self ""Utopia System""",Music,2018-09-07 00:00:00,Aragon Ballroom,None,4800,"1106 W Lawrence Ave, Chicago, IL 60640",4800.0,Event Space,41.969436,-87.658038,2018-09-07
2,5 Seconds of Summer / The Aces,Music,2018-09-08 00:00:00,Aragon Ballroom,None,4800,"1106 W Lawrence Ave, Chicago, IL 60640",4800.0,Event Space,41.969436,-87.658038,2018-09-08
3,SIGRID W/ HOUSES,Music,2019-08-01 00:00:00,Thalia Hall,Medium,800,"1807 S Allport St, Chicago, IL 60608",800.0,Music,41.857681,-87.657392,2019-08-01
4,JUDAH & THE LION W/ THE BAND CAMINO,Music,2019-08-02 00:00:00,Thalia Hall,Medium,800,"1807 S Allport St, Chicago, IL 60608",800.0,Music,41.857681,-87.657392,2019-08-02
...,...,...,...,...,...,...,...,...,...,...,...,...
920,Chicago Cubs vs Los Angeles Dodgers,Sports,2019-03-20 00:00:00,Wrigley Field,Large,41160,"1060 W Addison St, Chicago, IL 60613",41160.0,Sports,41.947588,-87.656125,2019-03-20
921,Chicago Cubs vs San Francisco Giants,Sports,2019-03-21 00:00:00,Wrigley Field,Large,41160,"1060 W Addison St, Chicago, IL 60613",41160.0,Sports,41.947588,-87.656125,2019-03-21
922,Chicago Cubs vs Los Angeles Angels,Sports,2019-06-03 00:00:00,Wrigley Field,Large,41160,"1060 W Addison St, Chicago, IL 60613",41160.0,Sports,41.947588,-87.656125,2019-06-03
923,14TH ANNUAL RACE TO WRIGLEY 5K CHARITY RUN PRE...,Sports,2019-04-27 00:00:00,Wrigley Field,Large,41160,"1060 W Addison St, Chicago, IL 60613",41160.0,Sports,41.947588,-87.656125,2019-04-27


In [36]:
data = pd.read_csv('data/crime_event_treated.csv')
data['date'] = pd.to_datetime(data.date)
data

,Unnamed: 0,id,case_number,date,block,iucr,primary_type,description,location_description,arrest,...,8to11_crime,9to12_crime,event_evening_crimes,event_3to6_crimes,event_4to7_crimes,event_5to8_crimes,event_6to9_crimes,event_7to10_crimes,event_8to11_crimes,event_9to12_crimes
0,0,11393720,JB367241,2018-07-27 00:00:00,047XX N BROADWAY,1305,CRIMINAL DAMAGE,CRIMINAL DEFACEMENT,BANK,False,...,0,0,0,0,0,0,0,0,0,0
1,1,11394961,JB368503,2018-07-27 00:00:00,001XX W HUBBARD ST,0870,THEFT,POCKET-PICKING,TAVERN/LIQUOR STORE,False,...,0,0,0,0,0,0,0,0,0,0
2,2,11480254,JB480430,2018-07-27 00:00:00,012XX S MICHIGAN AVE,1153,DECEPTIVE PRACTICE,FINANCIAL IDENTITY THEFT OVER $ 300,RESIDENCE,False,...,0,0,0,0,0,0,0,0,0,0
3,3,11395054,JB368869,2018-07-27 00:01:00,040XX N BROADWAY,0620,BURGLARY,UNLAWFUL ENTRY,APARTMENT,False,...,0,0,0,0,0,0,0,0,0,0
4,4,11395057,JB368870,2018-07-27 00:01:00,040XX N BROADWAY,1150,DECEPTIVE PRACTICE,CREDIT CARD FRAUD,APARTMENT,False,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103490,103490,11790084,JC387494,2019-08-10 23:45:00,001XX S HALSTED ST,0870,THEFT,POCKET-PICKING,BAR OR TAVERN,False,...,1,1,0,0,0,0,0,0,0,0
103491,103491,11790431,JC387911,2019-08-10 23:59:00,040XX S FEDERAL ST,1320,CRIMINAL DAMAGE,TO VEHICLE,STREET,False,...,1,1,0,0,0,0,0,0,0,0
103492,103492,11790428,JC387941,2019-08-11 00:00:00,031XX N SHEFFIELD AVE,1310,CRIMINAL DAMAGE,TO PROPERTY,RESIDENCE,False,...,0,0,0,0,0,0,0,0,0,0
103493,103493,11791865,JC389527,2019-08-11 00:00:00,009XX W BELMONT AVE,0890,THEFT,FROM BUILDING,BAR OR TAVERN,False,...,0,0,0,0,0,0,0,0,0,0


For the purpose of the following analysis, we will not use any crimes that did not occur within the bandwidth of at least one venue. These crimes could be useful for training a predictive model to estimate the counterfactual of crimes at a venue, but otherwise should not be included in the analysis. 

In [37]:
relevant_crimes = data[data.near_any_venue==1]

In [38]:
len(relevant_crimes)

48197

Simple difference in means: $\bar{Y}(D_i>0)-\bar{Y}(D_i=0)$. By "mean", we mean the average number of crimes per time period under consideration. 

First we'll do this with days. There are 381 days and 26 venues in our dataset. So we calculate the average number of crimes near venues on days that they have events, minus the average number of crimes near venues on days that they don't have events.

In [6]:
venuedays_with_events = len(events)
venuedays_without_events = 381*26 - venuedays_with_events

avg_crime_per_day_events = len(relevant_crimes[relevant_crimes.event_day>0])/venuedays_with_events
avg_crime_per_day_noevents = len(relevant_crimes[relevant_crimes.event_day==0])/venuedays_without_events

print(f"Average crime near venues on days with events: {avg_crime_per_day_events}")
print(f"Average crime near venues on days without events: {avg_crime_per_day_noevents}")
print(f"Naive estimated effect of events on crime: {avg_crime_per_day_events-avg_crime_per_day_noevents}")

Average crime near venues on days with events: 10.518918918918919
Average crime near venues on days without events: 4.283153323683331
Naive estimated effect of events on crime: 6.235765595235588


So there are many more crimes near our venues on days where they have events than on days where they don't. This is a fact, but doesn't necessarily imply causation. As a causal estimate this is likely to be biased upwards for many reasons, primarily due to selection (see technical log).

In [7]:
# repeat but for evening crimes

# note that the number of venue-days and venue-evenings with/without events is the same
avg_crime_per_evening_events = len(relevant_crimes[relevant_crimes.event_evening>0])/venuedays_with_events
avg_crime_per_evening_noevents = len(relevant_crimes[relevant_crimes.event_evening==0])/venuedays_without_events

print(f"Average crime near venues on evenings with events: {avg_crime_per_evening_events}")
print(f"Average crime near venues on evenings without events: {avg_crime_per_evening_noevents}")
print(f"Naive estimated effect of events on evening crime: {avg_crime_per_evening_events-avg_crime_per_evening_noevents}")

Average crime near venues on evenings with events: 2.982702702702703
Average crime near venues on evenings without events: 5.059347511412983
Naive estimated effect of events on evening crime: -2.07664480871028


When we only consider crimes that occurred in the evening, though, the estimated effect reverses! Another fact appears to be that there are _less_ evening crimes near our venues on days where there are events than on days where there aren't events. 

Now we'll move on to regression- and prediction-based approaches. Our outcome is the number of crimes per venue and date/time, so we will need to modify our dataset: each observation should be per venue, per date/time. 

Our first version of this will be for the day and evening time bandwidths. Our dataset will contain 26*380 observations: one per venue per day. Each observation's attributes will be venue (including as dummy variable), date, number of day crimes, number of evening crimes, event binary.

In [39]:
relevant_crimes.date.max()

Timestamp('2019-08-11 00:00:00')

In [40]:
date_list = [dt.date.fromisoformat('2019-08-11') - dt.timedelta(days=i) for i in range(381)]
locations = list(set(events.Location))

In [41]:
# source: https://stackoverflow.com/questions/25634489/get-all-combinations-of-elements-from-two-lists
relevant_crimes_daily = pd.DataFrame(list(product(date_list, locations)), columns=['date','Location'])
relevant_crimes_daily['date'] = pd.to_datetime(relevant_crimes_daily['date'])
relevant_crimes_daily

,date,Location
0,2019-08-11,The Chicago Theatre
1,2019-08-11,Main Hall in UIC Dorin Forum
2,2019-08-11,Arie Crown Theater
3,2019-08-11,Lincoln Park Zoo
4,2019-08-11,Cinespace Chicago Film Studios
...,...,...
9901,2018-07-27,Concord Music Hall
9902,2018-07-27,Soldier Field
9903,2018-07-27,Credit Union 1 Arena
9904,2018-07-27,Huntington Bank Pavilion


In [42]:
relevant_crimes_daily = pd.merge(relevant_crimes_daily, events[["date","Location","Category"]], on=['date','Location'],how='left')
relevant_crimes_daily

,date,Location,Category
0,2019-08-11,The Chicago Theatre,NaN
1,2019-08-11,Main Hall in UIC Dorin Forum,NaN
2,2019-08-11,Arie Crown Theater,NaN
3,2019-08-11,Lincoln Park Zoo,NaN
4,2019-08-11,Cinespace Chicago Film Studios,NaN
...,...,...,...
9932,2018-07-27,Concord Music Hall,NaN
9933,2018-07-27,Soldier Field,NaN
9934,2018-07-27,Credit Union 1 Arena,NaN
9935,2018-07-27,Huntington Bank Pavilion,NaN


Note that our dataframe increased in size. This indicates that there are actually events that occur at the same venue on the same day. We'll remove and ignore those. (Also, this means that our estimates above were actually slightly off!)

In future analysis it will be valuable to look at a breakdown by event type, as there could be heterogenous treatment effect by type. But we will not do that here.

In [43]:
relevant_crimes_daily["event"] = np.where(
    relevant_crimes_daily.Category.isna(), 0, 1)
relevant_crimes_daily = relevant_crimes_daily.drop(columns=['Category']).drop_duplicates()
relevant_crimes_daily

,date,Location,event
0,2019-08-11,The Chicago Theatre,0
1,2019-08-11,Main Hall in UIC Dorin Forum,0
2,2019-08-11,Arie Crown Theater,0
3,2019-08-11,Lincoln Park Zoo,0
4,2019-08-11,Cinespace Chicago Film Studios,0
...,...,...,...
9932,2018-07-27,Concord Music Hall,0
9933,2018-07-27,Soldier Field,0
9934,2018-07-27,Credit Union 1 Arena,0
9935,2018-07-27,Huntington Bank Pavilion,0


In [44]:
relevant_crimes_daily.event.value_counts()

0    9013
1     893
Name: event, dtype: int64

In [45]:
venuedays_with_events = len(relevant_crimes_daily[relevant_crimes_daily.event==1])
venuedays_without_events = len(relevant_crimes_daily[relevant_crimes_daily.event==0])

avg_crime_per_day_events = len(relevant_crimes[relevant_crimes.event_day>0])/venuedays_with_events
avg_crime_per_day_noevents = len(relevant_crimes[relevant_crimes.event_day==0])/venuedays_without_events

print("Days:")
print(f"Average crime near venues on days with events: {avg_crime_per_day_events}")
print(f"Average crime near venues on days without events: {avg_crime_per_day_noevents}")
print(f"Naive estimated effect of events on crime: {avg_crime_per_day_events-avg_crime_per_day_noevents}")

# note that the number of venue-days and venue-evenings with/without events is the same
avg_crime_per_evening_events = len(relevant_crimes[relevant_crimes.event_evening>0])/venuedays_with_events
avg_crime_per_evening_noevents = len(relevant_crimes[relevant_crimes.event_evening==0])/venuedays_without_events

print("\nEvenings:")
print(f"Average crime near venues on days with events: {avg_crime_per_evening_events}")
print(f"Average crime near venues on days without events: {avg_crime_per_evening_noevents}")
print(f"Naive estimated effect of events on crime: {avg_crime_per_evening_events-avg_crime_per_evening_noevents}")

Days:
Average crime near venues on days with events: 10.89585666293393
Average crime near venues on days without events: 4.267946299789194
Naive estimated effect of events on crime: 6.6279103631447365

Evenings:
Average crime near venues on days with events: 3.089585666293393
Average crime near venues on days without events: 5.041384666592699
Naive estimated effect of events on crime: -1.9517990002993062


In [46]:
for location in locations:
    relevant_crimes_daily[location] = np.where(
        relevant_crimes_daily.Location==location,
        1,0)
relevant_crimes_daily

,date,Location,event,The Chicago Theatre,Main Hall in UIC Dorin Forum,Arie Crown Theater,Lincoln Park Zoo,Cinespace Chicago Film Studios,Millennium Park,Petrillo Music Shell,...,Riviera Theatre,Gallagher Way,Guaranteed Rate Field,Sheraton Grand Chicago,Wrigley Field,Concord Music Hall,Soldier Field,Credit Union 1 Arena,Huntington Bank Pavilion,Aragon Ballroom
0,2019-08-11,The Chicago Theatre,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2019-08-11,Main Hall in UIC Dorin Forum,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2019-08-11,Arie Crown Theater,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2019-08-11,Lincoln Park Zoo,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2019-08-11,Cinespace Chicago Film Studios,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9932,2018-07-27,Concord Music Hall,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
9933,2018-07-27,Soldier Field,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
9934,2018-07-27,Credit Union 1 Arena,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
9935,2018-07-27,Huntington Bank Pavilion,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [47]:
relevant_crimes["day"] = relevant_crimes.date.dt.date
crimes_by_date = relevant_crimes.groupby(by=['day']).sum()[locations].reset_index()
crimes_by_date["day"] = pd.to_datetime(crimes_by_date["day"])
crimes_by_date.rename(columns={'day':'date'}, inplace=True)
crimes_by_date

/project/graziul/ra/drewkeller/geoconda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,date,The Chicago Theatre,Main Hall in UIC Dorin Forum,Arie Crown Theater,Lincoln Park Zoo,Cinespace Chicago Film Studios,Millennium Park,Petrillo Music Shell,Grant Park,Field Museum,...,Riviera Theatre,Gallagher Way,Guaranteed Rate Field,Sheraton Grand Chicago,Wrigley Field,Concord Music Hall,Soldier Field,Credit Union 1 Arena,Huntington Bank Pavilion,Aragon Ballroom
0,2018-07-27,43,2,10,4,6,33,20,13,8,...,8,10,4,32,14,6,10,5,1,8
1,2018-07-28,34,4,3,6,8,27,23,16,8,...,11,12,11,34,12,9,13,9,2,11
2,2018-07-29,45,6,8,6,6,32,22,5,3,...,3,21,5,33,21,11,6,6,0,3
3,2018-07-30,43,8,8,6,8,38,35,14,10,...,6,12,3,33,13,4,16,4,0,6
4,2018-07-31,41,4,4,5,5,33,19,12,9,...,12,6,4,33,8,7,12,10,0,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
376,2019-08-07,34,4,4,7,7,30,23,11,8,...,10,9,7,30,7,5,11,6,1,9
377,2019-08-08,42,7,6,2,5,38,25,14,8,...,16,8,6,29,11,7,13,5,0,17
378,2019-08-09,34,9,6,6,6,31,21,9,4,...,8,11,4,33,9,7,6,13,0,7
379,2019-08-10,41,5,7,3,8,29,15,7,5,...,10,17,6,33,16,6,9,11,0,10


In [82]:
# check
data[(data["Arie Crown Theater"]==1) & (data.date.dt.date == dt.date.fromisoformat('2018-07-29'))]  # should be 8 per above

,Unnamed: 0,id,case_number,date,block,iucr,primary_type,description,location_description,arrest,...,8to11_crime,9to12_crime,event_evening_crimes,event_3to6_crimes,event_4to7_crimes,event_5to8_crimes,event_6to9_crimes,event_7to10_crimes,event_8to11_crimes,event_9to12_crimes
685,685,11395661,JB369820,2018-07-29 02:30:00,017XX S STATE ST,1210,DECEPTIVE PRACTICE,THEFT OF LABOR/SERVICES,TAXICAB,True,...,0,0,0,0,0,0,0,0,0,0
758,758,11400175,JB370130,2018-07-29 11:20:00,005XX W 18TH ST,0580,STALKING,SIMPLE,RESIDENCE,False,...,0,0,0,0,0,0,0,0,0,0
812,812,11396812,JB370327,2018-07-29 14:26:00,028XX S STATE ST,0486,BATTERY,DOMESTIC BATTERY SIMPLE,APARTMENT,False,...,0,0,0,0,0,0,0,0,0,0
875,875,11396606,JB370949,2018-07-29 18:15:00,025XX S MICHIGAN AVE,0460,BATTERY,SIMPLE,HOSPITAL BUILDING/GROUNDS,False,...,0,0,0,0,0,0,0,0,0,0
880,880,11396515,JB370925,2018-07-29 18:30:00,020XX S WABASH AVE,0810,THEFT,OVER $500,STREET,False,...,0,0,0,0,0,0,0,0,0,0
902,902,11396693,JB371175,2018-07-29 19:45:00,026XX S NORMAL AVE,0910,MOTOR VEHICLE THEFT,AUTOMOBILE,STREET,False,...,0,0,0,0,0,0,0,0,0,0
934,934,11396490,JB370844,2018-07-29 21:36:00,021XX S TAN CT,0330,ROBBERY,AGGRAVATED,SIDEWALK,True,...,1,1,0,0,0,0,0,0,0,0
948,948,11396519,JB370843,2018-07-29 22:07:00,023XX S PRINCETON AVE,0340,ROBBERY,ATTEMPT: STRONGARM-NO WEAPON,RESIDENTIAL YARD (FRONT/BACK),True,...,1,1,0,0,0,0,0,0,0,0


In [83]:
# assign proper crimes from crimes_by_date to relevant_crimes_daily
relevant_crimes_daily = pd.merge(relevant_crimes_daily, crimes_by_date, on=['date'], how='left')
relevant_crimes_daily

,date,Location,event,The Chicago Theatre_x,Main Hall in UIC Dorin Forum_x,Arie Crown Theater_x,Lincoln Park Zoo_x,Cinespace Chicago Film Studios_x,Millennium Park_x,Petrillo Music Shell_x,...,Riviera Theatre_y,Gallagher Way_y,Guaranteed Rate Field_y,Sheraton Grand Chicago_y,Wrigley Field_y,Concord Music Hall_y,Soldier Field_y,Credit Union 1 Arena_y,Huntington Bank Pavilion_y,Aragon Ballroom_y
0,2019-08-11,The Chicago Theatre,0,1,0,0,0,0,0,0,...,0,2,0,0,3,0,0,0,0,0
1,2019-08-11,Main Hall in UIC Dorin Forum,0,0,1,0,0,0,0,0,...,0,2,0,0,3,0,0,0,0,0
2,2019-08-11,Arie Crown Theater,0,0,0,1,0,0,0,0,...,0,2,0,0,3,0,0,0,0,0
3,2019-08-11,Lincoln Park Zoo,0,0,0,0,1,0,0,0,...,0,2,0,0,3,0,0,0,0,0
4,2019-08-11,Cinespace Chicago Film Studios,0,0,0,0,0,1,0,0,...,0,2,0,0,3,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9901,2018-07-27,Concord Music Hall,0,0,0,0,0,0,0,0,...,8,10,4,32,14,6,10,5,1,8
9902,2018-07-27,Soldier Field,0,0,0,0,0,0,0,0,...,8,10,4,32,14,6,10,5,1,8
9903,2018-07-27,Credit Union 1 Arena,0,0,0,0,0,0,0,0,...,8,10,4,32,14,6,10,5,1,8
9904,2018-07-27,Huntington Bank Pavilion,0,0,0,0,0,0,0,0,...,8,10,4,32,14,6,10,5,1,8


In [18]:
relevant_crimes_daily["day_crimes"] = 0
for location in locations:
    relevant_crimes_daily["day_crimes"] = np.where(
        relevant_crimes_daily.Location==location,
        relevant_crimes_daily[location+'_y'],
        relevant_crimes_daily["day_crimes"])
relevant_crimes_daily

,date,Location,event,The Chicago Theatre_x,Main Hall in UIC Dorin Forum_x,Arie Crown Theater_x,Lincoln Park Zoo_x,Cinespace Chicago Film Studios_x,Millennium Park_x,Petrillo Music Shell_x,...,Gallagher Way_y,Guaranteed Rate Field_y,Sheraton Grand Chicago_y,Wrigley Field_y,Concord Music Hall_y,Soldier Field_y,Credit Union 1 Arena_y,Huntington Bank Pavilion_y,Aragon Ballroom_y,event_day_crimes
0,2019-08-11,The Chicago Theatre,0,1,0,0,0,0,0,0,...,2,0,0,3,0,0,0,0,0,0
1,2019-08-11,Main Hall in UIC Dorin Forum,0,0,1,0,0,0,0,0,...,2,0,0,3,0,0,0,0,0,0
2,2019-08-11,Arie Crown Theater,0,0,0,1,0,0,0,0,...,2,0,0,3,0,0,0,0,0,0
3,2019-08-11,Lincoln Park Zoo,0,0,0,0,1,0,0,0,...,2,0,0,3,0,0,0,0,0,0
4,2019-08-11,Cinespace Chicago Film Studios,0,0,0,0,0,1,0,0,...,2,0,0,3,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9901,2018-07-27,Concord Music Hall,0,0,0,0,0,0,0,0,...,10,4,32,14,6,10,5,1,8,6
9902,2018-07-27,Soldier Field,0,0,0,0,0,0,0,0,...,10,4,32,14,6,10,5,1,8,10
9903,2018-07-27,Credit Union 1 Arena,0,0,0,0,0,0,0,0,...,10,4,32,14,6,10,5,1,8,5
9904,2018-07-27,Huntington Bank Pavilion,0,0,0,0,0,0,0,0,...,10,4,32,14,6,10,5,1,8,1


In [19]:
relevant_crimes_daily = relevant_crimes_daily[['date', 'Location', 'event','day_crimes', 'Thalia Hall_x',
       'Huntington Bank Pavilion_x', 'Union Station_x',
       'Cinespace Chicago Film Studios_x', 'Lincoln Park Zoo_x',
       'Chicago Symphony Orchestra_x', 'Petrillo Music Shell_x',
       'Millennium Park_x', 'Gallagher Way_x', 'Wrigley Field_x',
       'Concord Music Hall_x', 'Sheraton Grand Chicago_x',
       'The Chicago Theatre_x', 'Aragon Ballroom_x', 'Auditorium Theatre_x',
       'Museum of Science and Industry_x', 'Field Museum_x', 'Grant Park_x',
       'Main Hall in UIC Dorin Forum_x', 'Arie Crown Theater_x',
       'Soldier Field_x', 'Civic Opera House_x', 'United Center_x',
       'Guaranteed Rate Field_x', 'Credit Union 1 Arena_x',
       'Riviera Theatre_x']]

In [20]:
for column in relevant_crimes_daily.columns:
    if column.endswith('_x'):
        relevant_crimes_daily = relevant_crimes_daily.rename(columns={column:column[:-2]})
relevant_crimes_daily

,date,Location,event,event_day_crimes,Thalia Hall,Huntington Bank Pavilion,Union Station,Cinespace Chicago Film Studios,Lincoln Park Zoo,Chicago Symphony Orchestra,...,Field Museum,Grant Park,Main Hall in UIC Dorin Forum,Arie Crown Theater,Soldier Field,Civic Opera House,United Center,Guaranteed Rate Field,Credit Union 1 Arena,Riviera Theatre
0,2019-08-11,The Chicago Theatre,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2019-08-11,Main Hall in UIC Dorin Forum,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,2019-08-11,Arie Crown Theater,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
3,2019-08-11,Lincoln Park Zoo,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,2019-08-11,Cinespace Chicago Film Studios,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9901,2018-07-27,Concord Music Hall,0,6,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9902,2018-07-27,Soldier Field,0,10,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
9903,2018-07-27,Credit Union 1 Arena,0,5,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
9904,2018-07-27,Huntington Bank Pavilion,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [21]:
# now make this a function and add other crime types

def add_crimes(relevant_crimes_daily,relevant_crimes,crime_type):
    crimes_by_date = relevant_crimes[relevant_crimes[crime_type]>0].groupby(by=['day']).sum()[locations].reset_index()
    crimes_by_date["day"] = pd.to_datetime(crimes_by_date["day"])
    crimes_by_date.rename(columns={'day':'date'}, inplace=True)
    relevant_crimes_daily = pd.merge(relevant_crimes_daily, crimes_by_date, on=['date'], how='left')
    relevant_crimes_daily = relevant_crimes_daily.fillna(0)
    relevant_crimes_daily[crime_type] = 0
    for location in locations:
        relevant_crimes_daily[crime_type] = np.where(
            relevant_crimes_daily.Location==location,
            relevant_crimes_daily[location+'_y'],
            relevant_crimes_daily[crime_type])
    relevant_crimes_daily = relevant_crimes_daily.drop(columns=['Petrillo Music Shell_y', 'Grant Park_y', 'Guaranteed Rate Field_y',
       'United Center_y', 'Lincoln Park Zoo_y', 'Riviera Theatre_y',
       'Thalia Hall_y', 'Millennium Park_y', 'Field Museum_y',
       'Concord Music Hall_y', 'Wrigley Field_y', 'Auditorium Theatre_y',
       'Huntington Bank Pavilion_y', 'Union Station_y',
       'Main Hall in UIC Dorin Forum_y', 'Museum of Science and Industry_y',
       'Credit Union 1 Arena_y', 'Civic Opera House_y', 'Soldier Field_y',
       'Sheraton Grand Chicago_y', 'Cinespace Chicago Film Studios_y',
       'Gallagher Way_y', 'Arie Crown Theater_y', 'The Chicago Theatre_y',
       'Chicago Symphony Orchestra_y', 'Aragon Ballroom_y',])
    for column in relevant_crimes_daily.columns:
        if column.endswith('_x'):
            relevant_crimes_daily = relevant_crimes_daily.rename(columns={column:column[:-2]})
    
    return relevant_crimes_daily

event_types = [
    "day_crimes","evening_crimes","3to6_crimes","4to7_crimes","5to8_crimes",
    "6to9_crimes","7to10_crimes","8to11_crimes","9to12_crimes"
    ]
for crime_type in event_types[1:]:
    relevant_crimes_daily = add_crimes(relevant_crimes_daily,relevant_crimes,crime_type)
relevant_crimes_daily

,date,Location,event,event_day_crimes,Thalia Hall,Huntington Bank Pavilion,Union Station,Cinespace Chicago Film Studios,Lincoln Park Zoo,Chicago Symphony Orchestra,...,Credit Union 1 Arena,Riviera Theatre,event_evening_crimes,event_3to6_crimes,event_4to7_crimes,event_5to8_crimes,event_6to9_crimes,event_7to10_crimes,event_8to11_crimes,event_9to12_crimes
0,2019-08-11,The Chicago Theatre,0,0,0,0,0,0,0,0,...,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2019-08-11,Main Hall in UIC Dorin Forum,0,0,0,0,0,0,0,0,...,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2019-08-11,Arie Crown Theater,0,0,0,0,0,0,0,0,...,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2019-08-11,Lincoln Park Zoo,0,0,0,0,0,0,1,0,...,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2019-08-11,Cinespace Chicago Film Studios,0,0,0,0,0,1,0,0,...,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9901,2018-07-27,Concord Music Hall,0,6,0,0,0,0,0,0,...,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9902,2018-07-27,Soldier Field,0,10,0,0,0,0,0,0,...,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9903,2018-07-27,Credit Union 1 Arena,0,5,0,0,0,0,0,0,...,1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9904,2018-07-27,Huntington Bank Pavilion,0,1,0,1,0,0,0,0,...,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Now we'll first try a simple linear regression of daily crimes on whether there was an event nearby. This could recover an unbiased estimate of the effect of events on crimes if venues had been completely randomly assigned to have events - which they obviously did not.

In [22]:
X = relevant_crimes_daily.event
Y = relevant_crimes_daily.day_crimes

# sklearn version
#day_regression = linear_model.LinearRegression()
#day_regression.fit(X,Y)

print('Same day - simple regression:')
sm.OLS(Y, X).fit().summary()

Same day - simple regression:


<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:       event_day_crimes   R-squared (uncentered):                   0.042
Model:                            OLS   Adj. R-squared (uncentered):              0.041
Method:                 Least Squares   F-statistic:                              429.9
Date:                Mon, 20 Jun 2022   Prob (F-statistic):                    1.64e-93
Time:                        17:59:55   Log-Likelihood:                         -42573.
No. Observations:                9906   AIC:                                  8.515e+04
Df Residuals:                    9905   BIC:                                  8.515e+04
Df Model:                           1                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
event         12.3449      0.595     20.733      0.000      11.178      13.512
==============================================================================
Omnibus:                     3404.642   Durbin-Watson:                   1.165
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            15920.253
Skew:                           1.611   Prob(JB):                         0.00
Kurtosis:                       8.309   Cond. No.                         1.00
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

This estimates that having an event causes, on average, 12 more crimes that day within a 2km square around the venue. However, this is certainly biased due to our lack of actually being able to randomize events; any factors that both make crimes and events more likely would induce upward bias on our estimate (for example, day of the week, or venue type).

To try to somewhat account for this we will now repeat the regression with venue fixed effects and then venue and date fixed effects.

In [23]:
X = relevant_crimes_daily.drop(columns=['Location','date']+event_types)
Y = relevant_crimes_daily.day_crimes

print('Same-day crime - venue fixed effects:')
sm.OLS(Y, X).fit().summary()

Same-day crime - venue fixed effects:


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:       event_day_crimes   R-squared:                       0.759
Model:                            OLS   Adj. R-squared:                  0.759
Method:                 Least Squares   F-statistic:                     1200.
Date:                Mon, 20 Jun 2022   Prob (F-statistic):               0.00
Time:                        17:59:55   Log-Likelihood:                -32022.
No. Observations:                9906   AIC:                         6.410e+04
Df Residuals:                    9879   BIC:                         6.429e+04
Df Model:                          26                                         
Covariance Type:            nonrobust                                         
==================================================================================================
                                     coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------
event                              0.1228      0.227      0.540      0.589      -0.323       0.568
Thalia Hall                        4.8968      0.319     15.370      0.000       4.272       5.521
Huntington Bank Pavilion           0.3607      0.315      1.146      0.252      -0.257       0.978
Union Station                     16.1913      0.315     51.463      0.000      15.575      16.808
Cinespace Chicago Film Studios     6.3330      0.315     20.129      0.000       5.716       6.950
Lincoln Park Zoo                   3.8767      0.315     12.316      0.000       3.260       4.494
Chicago Symphony Orchestra        27.7237      0.316     87.708      0.000      27.104      28.343
Petrillo Music Shell              23.3553      0.315     74.230      0.000      22.739      23.972
Millennium Park                   31.8953      0.316    100.810      0.000      31.275      32.516
Gallagher Way                     10.0755      0.317     31.825      0.000       9.455      10.696
Wrigley Field                     10.4201      0.315     33.085      0.000       9.803      11.037
Concord Music Hall                 6.5276      0.323     20.213      0.000       5.895       7.161
Sheraton Grand Chicago            32.7894      0.315    104.218      0.000      32.173      33.406
The Chicago Theatre               38.5281      0.315    122.247      0.000      37.910      39.146
Aragon Ballroom                    8.7254      0.317     27.543      0.000       8.104       9.346
Auditorium Theatre                24.2813      0.315     77.103      0.000      23.664      24.899
Museum of Science and Industry     3.1720      0.315     10.079      0.000       2.555       3.789
Field Museum                       5.2510      0.315     16.690      0.000       4.634       5.868
Grant Park                        10.6982      0.315     33.999      0.000      10.081      11.315
Main Hall in UIC Dorin Forum       5.6191      0.315     17.860      0.000       5.002       6.236
Arie Crown Theater                 5.5069      0.315     17.502      0.000       4.890       6.124
Soldier Field                      8.4549      0.315     26.830      0.000       7.837       9.073
Civic Opera House                 31.2872      0.315     99.294      0.000      30.670      31.905
United Center                      6.9210      0.324     21.349      0.000       6.286       7.557
Guaranteed Rate Field              4.3495      0.315     13.819      0.000       3.733       4.966
Credit Union 1 Arena               6.5718      0.316     20.822      0.000       5.953       7.190
Riviera Theatre                    8.7953      0.316     27.793      0.000       8.175       9.416
==============================================================================
Omnibus:                     9107.

In [24]:
time_dummies = relevant_crimes_daily.copy(deep=True).drop(columns='Location')
time_dummies["date"]=time_dummies.date.astype(str)
time_dummies = pd.get_dummies(time_dummies)
time_dummies

,event,event_day_crimes,Thalia Hall,Huntington Bank Pavilion,Union Station,Cinespace Chicago Film Studios,Lincoln Park Zoo,Chicago Symphony Orchestra,Petrillo Music Shell,Millennium Park,...,date_2019-08-02,date_2019-08-03,date_2019-08-04,date_2019-08-05,date_2019-08-06,date_2019-08-07,date_2019-08-08,date_2019-08-09,date_2019-08-10,date_2019-08-11
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9901,0,6,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9902,0,10,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9903,0,5,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9904,0,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [32]:
X = time_dummies.drop(columns=event_types+locations)
Y = time_dummies.day_crimes

print('Same-day crime - date fixed effects:')
sm.OLS(Y, X).fit().summary()

Same-day crime - date fixed effects:


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:       event_day_crimes   R-squared:                       0.066
Model:                            OLS   Adj. R-squared:                  0.029
Method:                 Least Squares   F-statistic:                     1.765
Date:                Mon, 20 Jun 2022   Prob (F-statistic):           1.61e-17
Time:                        18:01:58   Log-Likelihood:                -38741.
No. Observations:                9906   AIC:                         7.825e+04
Df Residuals:                    9524   BIC:                         8.100e+04
Df Model:                         381                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
event              -1.1094      0.446     -2.489      0.013      -1.983      -0.236
date_2018-07-27    14.2308      2.417      5.888      0.000       9.493      18.969
date_2018-07-28    14.0042      2.417      5.794      0.000       9.266      18.742
date_2018-07-29    13.8077      2.417      5.713      0.000       9.070      18.546
date_2018-07-30    15.7692      2.417      6.524      0.000      11.031      20.507
date_2018-07-31    14.6538      2.417      6.063      0.000       9.916      19.392
date_2018-08-01    17.3203      2.418      7.164      0.000      12.581      22.059
date_2018-08-02    23.8546      2.417      9.868      0.000      19.116      28.593
date_2018-08-03    27.9399      2.418     11.555      0.000      23.200      32.680
date_2018-08-04    26.7818      2.418     11.078      0.000      22.043      31.521
date_2018-08-05    22.6581      2.417      9.374      0.000      17.920      27.396
date_2018-08-06    14.2734      2.417      5.905      0.000       9.535      19.012
date_2018-08-07    13.7392      2.417      5.684      0.000       9.001      18.478
date_2018-08-08    14.0811      2.417      5.826      0.000       9.343      18.819
date_2018-08-09    11.3546      2.417      4.697      0.000       6.616      16.093
date_2018-08-10    17.5084      2.417      7.243      0.000      12.770      22.247
date_2018-08-11    13.8203      2.418      5.716      0.000       9.081      18.559
date_2018-08-12    12.7776      2.417      5.286      0.000       8.039      17.516
date_2018-08-13    14.0469      2.417      5.811      0.000       9.308      18.785
date_2018-08-14    15.1749      2.419      6.274      0.000      10.434      19.916
date_2018-08-15    14.1581      2.417      5.857      0.000       9.420      18.896
date_2018-08-16    16.3546      2.417      6.766      0.000      11.616      21.093
date_2018-08-17    12.6581      2.417      5.237      0.000       7.920      17.396
date_2018-08-18    15.7861      2.418      6.528      0.000      11.046      20.526
date_2018-08-19    12.2007      2.417      5.047      0.000       7.462      16.939
date_2018-08-20    12.0811      2.417      4.998      0.000       7.343      16.819
date_2018-08-21    14.6238      2.417      6.050      0.000       9.885      19.362
date_2018-08-22    14.8119      2.417      6.128      0.000      10.074      19.550
date_2018-08-23    14.7692      2.417      6.110      0.000      10.031      19.507
date_2018-08-24    13.9784      2.418      5.781      0.000       9.238      18.718
date_2018-08-25    12.4399      2.418      5.145      0.000       7.700      17.180
date_2018-08-26    11.5769      2.417      4.790      0.000       6.839      16.315
date_2018-08-27    15.1196      2.417      6.255      0.000      10.381      19.858
date_2018-08-28    12.9700      2.417      5.365      0.000

We shouldn't pay too much attention to these, though - considering all the crimes from the day is certainly much too high a time bandwidth that could hide any real effect.

We'll try the same thing with crimes from 6pm - midnight on the day of an event.

In [26]:
X = relevant_crimes_daily.event
Y = relevant_crimes_daily.evening_crimes

print('Evening - simple regression:')
sm.OLS(Y, X).fit().summary()

Evening - simple regression:


<class 'statsmodels.iolib.summary.Summary'>
"""
                                  OLS Regression Results                                 
=========================================================================================
Dep. Variable:     event_evening_crimes   R-squared (uncentered):                   0.190
Model:                              OLS   Adj. R-squared (uncentered):              0.190
Method:                   Least Squares   F-statistic:                              2319.
Date:                  Mon, 20 Jun 2022   Prob (F-statistic):                        0.00
Time:                          17:59:56   Log-Likelihood:                         -21658.
No. Observations:                  9906   AIC:                                  4.332e+04
Df Residuals:                      9905   BIC:                                  4.332e+04
Df Model:                             1                                                  
Covariance Type:              nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
event          3.4714      0.072     48.156      0.000       3.330       3.613
==============================================================================
Omnibus:                     6206.755   Durbin-Watson:                   1.631
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            70073.071
Skew:                           2.899   Prob(JB):                         0.00
Kurtosis:                      14.669   Cond. No.                         1.00
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [27]:
X = relevant_crimes_daily.drop(columns=['Location','date']+event_types)
Y = relevant_crimes_daily.evening_crimes

print('Evening - venue fixed effects:')
sm.OLS(Y, X).fit().summary()

Evening - venue fixed effects:


<class 'statsmodels.iolib.summary.Summary'>
"""
                             OLS Regression Results                             
================================================================================
Dep. Variable:     event_evening_crimes   R-squared:                       0.312
Model:                              OLS   Adj. R-squared:                  0.310
Method:                   Least Squares   F-statistic:                     172.2
Date:                  Mon, 20 Jun 2022   Prob (F-statistic):               0.00
Time:                          17:59:57   Log-Likelihood:                -19978.
No. Observations:                  9906   AIC:                         4.001e+04
Df Residuals:                      9879   BIC:                         4.020e+04
Df Model:                            26                                         
Covariance Type:              nonrobust                                         
==================================================================================================
                                     coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------
event                              2.9689      0.067     44.050      0.000       2.837       3.101
Thalia Hall                       -0.2819      0.094     -2.984      0.003      -0.467      -0.097
Huntington Bank Pavilion          -0.0901      0.093     -0.966      0.334      -0.273       0.093
Union Station                      0.8846      0.093      9.484      0.000       0.702       1.067
Cinespace Chicago Film Studios    -0.0052      0.093     -0.055      0.956      -0.188       0.178
Lincoln Park Zoo                  -0.0722      0.093     -0.774      0.439      -0.255       0.111
Chicago Symphony Orchestra         2.0383      0.094     21.751      0.000       1.855       2.222
Petrillo Music Shell               2.0896      0.093     22.402      0.000       1.907       2.272
Millennium Park                    2.3746      0.094     25.316      0.000       2.191       2.559
Gallagher Way                      0.1465      0.094      1.561      0.118      -0.037       0.331
Wrigley Field                      0.4219      0.093      4.519      0.000       0.239       0.605
Concord Music Hall                -0.2158      0.096     -2.254      0.024      -0.403      -0.028
Sheraton Grand Chicago             2.0317      0.093     21.781      0.000       1.849       2.215
The Chicago Theatre                2.5616      0.093     27.415      0.000       2.378       2.745
Aragon Ballroom                    0.2544      0.094      2.709      0.007       0.070       0.439
Auditorium Theatre                 1.8365      0.093     19.670      0.000       1.654       2.020
Museum of Science and Industry    -0.0568      0.093     -0.608      0.543      -0.240       0.126
Field Museum                       0.2916      0.093      3.126      0.002       0.109       0.474
Grant Park                         0.8458      0.093      9.067      0.000       0.663       1.029
Main Hall in UIC Dorin Forum       0.1733      0.093      1.858      0.063      -0.010       0.356
Arie Crown Theater                -0.0152      0.093     -0.163      0.871      -0.198       0.168
Soldier Field                      0.4198      0.093      4.493      0.000       0.237       0.603
Civic Opera House                  2.1808      0.093     23.345      0.000       1.998       2.364
United Center                     -0.0549      0.096     -0.571      0.568      -0.243       0.133
Guaranteed Rate Field             -0.0880      0.093     -0.943      0.346      -0.271       0.095
Credit Union 1 Arena               0.0244      0.094      0.261      0.794      -0.159       0.208
Riviera Theatre                    0.3144      0.094      3.351      0.001       0.130       0.498
==============================================================================
Omnibus:    

In [33]:
X = time_dummies.drop(columns=event_types+locations)
Y = time_dummies.evening_crimes

print('Evening crimes - date fixed effects:')
sm.OLS(Y, X).fit().summary()

Evening crimes - date fixed effects:


<class 'statsmodels.iolib.summary.Summary'>
"""
                             OLS Regression Results                             
================================================================================
Dep. Variable:     event_evening_crimes   R-squared:                       0.307
Model:                              OLS   Adj. R-squared:                  0.279
Method:                   Least Squares   F-statistic:                     11.07
Date:                  Mon, 20 Jun 2022   Prob (F-statistic):               0.00
Time:                          18:02:48   Log-Likelihood:                -20014.
No. Observations:                  9906   AIC:                         4.079e+04
Df Residuals:                      9524   BIC:                         4.354e+04
Df Model:                           381                                         
Covariance Type:              nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
event               2.4136      0.067     35.861      0.000       2.282       2.546
date_2018-07-27  9.486e-16      0.365    2.6e-15      1.000      -0.715       0.715
date_2018-07-28     1.0610      0.365      2.907      0.004       0.346       1.776
date_2018-07-29 -8.331e-17      0.365  -2.28e-16      1.000      -0.715       0.715
date_2018-07-30  2.215e-16      0.365   6.07e-16      1.000      -0.715       0.715
date_2018-07-31  1.173e-15      0.365   3.21e-15      1.000      -0.715       0.715
date_2018-08-01    -0.0477      0.365     -0.131      0.896      -0.763       0.668
date_2018-08-02     0.3143      0.365      0.861      0.389      -0.401       1.030
date_2018-08-03     0.0518      0.365      0.142      0.887      -0.664       0.767
date_2018-08-04     0.2984      0.365      0.817      0.414      -0.417       1.014
date_2018-08-05    -0.0928      0.365     -0.254      0.799      -0.808       0.623
date_2018-08-06     0.8687      0.365      2.380      0.017       0.153       1.584
date_2018-08-07     1.1990      0.365      3.285      0.001       0.483       1.914
date_2018-08-08    -0.0544      0.365     -0.149      0.882      -0.770       0.661
date_2018-08-09     1.1220      0.365      3.074      0.002       0.407       1.838
date_2018-08-10     1.8143      0.365      4.971      0.000       1.099       2.530
date_2018-08-11    -0.0093      0.365     -0.025      0.980      -0.725       0.706
date_2018-08-12     2.4297      0.365      6.657      0.000       1.714       3.145
date_2018-08-13     0.9297      0.365      2.547      0.011       0.214       1.645
date_2018-08-14     4.7666      0.365     13.052      0.000       4.051       5.482
date_2018-08-15     0.0226      0.365      0.062      0.951      -0.693       0.738
date_2018-08-16     2.7374      0.365      7.500      0.000       2.022       3.453
date_2018-08-17     2.1379      0.365      5.858      0.000       1.422       2.853
date_2018-08-18     3.3594      0.365      9.201      0.000       2.644       4.075
date_2018-08-19     1.7374      0.365      4.760      0.000       1.022       2.453
date_2018-08-20     0.2149      0.365      0.589      0.556      -0.501       0.930
date_2018-08-21     2.3528      0.365      6.446      0.000       1.637       3.068
date_2018-08-22     0.1379      0.365      0.378      0.705      -0.578       0.853
date_2018-08-23   2.28e-15      0.365   6.25e-15      1.000      -0.715       0.715
date_2018-08-24     1.3594      0.365      3.723      0.000       0.644       2.075
date_2018-08-25    -0.2175      0.365     -0.596      0.551      -0.933       0.498
date_2018-08-26 -2.954e-15      0.365  -8.09e-15      1.000      -0.715       0.715
date_2018-08-27     0.8302      0.365      2.275      0.023       0.115       1.546
date_2018-08-28     0.4297      0.365

With or without venue and time fixed effects, these models estimate that having an event on the same day increases evening crimes within a 2 km square by 2 to 3 per event (remember that we are treating multiple nearby events' effects as linearly additive). These esimates are both statistically and practically significant (remember that on average there are only 3 to 5 crimes logged per evening within a 2 km square of all the venues). Also, the likely-larger-than-ideal time bandwidth would probably attenuate this estimate from a real effect. On the other hand, these estimates are still very susceptible to bias from unobserved factors.

Finally, we'll estimate time and venue fixed effects regressions of crimes within each 3-hour time window on same-day events. Note that this is kinda basically the same as using a linear model to predict the crimes near each venue at each date and comparing that with the actual observed number of crimes (see technical log for more discussion).

In [29]:
results = {}
for event_type in event_types[2:]:
    X = time_dummies.drop(columns=event_types+locations)
    Y = time_dummies[event_type]
    model = sm.OLS(Y, X).fit()
    results[event_type] = [model.params[0],model.pvalues[0]]
print('Estimated effect of event on crimes by time period (and p values)')
results

Estimated effect of event on crimes by time period (and p values)


{'event_3to6_crimes': [1.8992907087881692, 9.713937474824331e-119],
 'event_4to7_crimes': [1.9060817948588968, 1.810271792570366e-141],
 'event_5to8_crimes': [1.8561798883243634, 2.5052254263619637e-169],
 'event_6to9_crimes': [1.7511444237637717, 7.068274245434048e-213],
 'event_7to10_crimes': [1.626138135721113, 5.762981298550751e-265],
 'event_8to11_crimes': [1.448060767644248, 2.041583225916071e-283],
 'event_9to12_crimes': [1.009004477086373, 4.0147560029775266e-258]}

These results are a bit strange. The estimates are of similar magnitude (events cause 1-2 additional crimes in 2 km square), reducing later in the evening, and are very statistically significant for all the periods from 3pm to midnight. 

Personally, I think these results are indicative of unaccounted-for selection: factors we are not including that both make crime at various times in the day and having an event more likely. 